In [ ]:
import pandas as pd

from autumn.core import inputs
from autumn.core.project import get_project

In [ ]:
pd.options.plotting.backend = "plotly"

In [ ]:
region = "northern_territory"
p = get_project("sm_sir", region, reload=True)
baseline_params = p.param_set.baseline
model = p.run_baseline_model(baseline_params)
derived_df = model.get_derived_outputs_df()

## Check population age/Indigenous distribution
Direct from the databases (as should be implemented in the model)

In [ ]:
age_groups = model._stratifications[0].strata
iso3 = baseline_params["country"]["iso3"]
region = baseline_params["population"]["region"]
year = baseline_params["population"]["year"]
age_pops = pd.Series(
    inputs.get_population_by_agegroup(age_groups, iso3, region, year),
    index=age_groups,
)
indigenous_age_pops = pd.Series(
    inputs.get_population_by_agegroup(age_groups, iso3, "NT_ABORIGINAL", year),
    index=age_groups,
)
non_indigenous_age_pops = age_pops - indigenous_age_pops

In [ ]:
pd.DataFrame({
    "indigenous": indigenous_age_pops,
    "non_indigenous": non_indigenous_age_pops,
}).plot()

Using the model's susceptible compartment itself (typically nearly all the starting population)

In [ ]:
age_pops = {}
for status in ["indigenous", "non_indigenous"]:
    age_pops[status] = {}
    for agegroup in age_groups:
        age_pops[status][agegroup] = model.initial_population[
            [c.idx for c in model.get_matching_compartments("susceptible", {"indigenous": status, "agegroup": agegroup})]
        ].sum()
pd.DataFrame(age_pops).plot()

In [ ]:
derived_df["cdr"].plot()

In [ ]:
def get_strata(model, strat_name):
    for strat in model._stratifications:
        if strat_name == strat.name:
            return strat.strata

In [ ]:
derived_df[[f"prop_immune_{stratum}" for stratum in get_strata(model, "immunity")]].plot.area()

In [ ]:
derived_df["prop_ever_infected"].plot.area()

In [ ]:
derived_df[[f"strain_propXstrain_{stratum}" for stratum in model._stratifications[2].strata]].plot.area()